In [ ]:
!pip install pandas gql requests_toolbelt

In [ ]:
import pandas as pd
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# GraphQL Examples

In this notebook we explore searching for metadata from the GraphQL API. The GraphQL API provides a method to programmatically extract a JSON representation of the meta data from the API. It has the advantage over REST that it can prevent under and over fetching data.

First we load some python dependancies that we will use as part of this notebook and set the variable `API_URL` to the location of the GraphQL API.

In [ ]:
# This is the location of the GraphQL API
API_URL = "https://mastapp.site/graphql"

Setup a `gql` client. This is used to query the graphql API

In [ ]:
# Select your transport with a defined url endpoint
transport = RequestsHTTPTransport(url=API_URL)
# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

## Querying Shots with GraphQL

With GraphQL you can query exactly what you want, rather than having to recieve the whole table from the database This is useful in cases where the whole table has many columns, but you are interested in just a subset of them.

The GraphQL endpoint is located at `/graphql`. You can find the documentation and an interactive query explorer at the URL below:

In [ ]:
print(f"GraphQL API Endpoint {API_URL}")

Unlike the REST API which uses HTTP `GET` requests to return data, with GraphQL we use HTTP `POST` to post our query to the API.

Here is a simple example of getting some shot data from the GraphQL API. We need to explicity state what information we want to return from the API. Here we are asking for:
 - the shot ID
 - the timestamp that the shot was taken
 - the preshot description
 - the divertor configuration

In [ ]:
# Write our GraphQL query.
query = gql("""
query {
    all_shots  {
        shots {
            shot_id
            timestamp
            preshot_description
            divertor_config
        }
    }
}
""")

# # Query the API and get a JSON response
result = client.execute(query)
shots = result['all_shots']['shots']
df = pd.DataFrame(shots)
df.head()

### Searching & Filtering Data
We can also supply query parameters to GraphQL, such as limiting the number of returned values or filtering by value. Here we are limiting the first 3 values and we are selcting only shots from the M9 campaign.

In [ ]:
# Write our GraphQL query.
query = gql("""
query ($campaign: String!) {
    all_shots (limit: 3, where: {campaign: {eq: $campaign}}) {
        shots {
            shot_id
            timestamp
            preshot_description
            divertor_config
        }
    }
}
""")
# Query the API and get a JSON response
result = client.execute(query, variable_values={"campaign": "M9"})
df = pd.DataFrame(result['all_shots']['shots'])
df.head()

### Nested Queries

One feature which makes GraphQL much more powerful than REST is that you may perform nested queries to gather different subsets of the data. For example, here we are going to query for all datasets with "AMC" in the name, and query for information about shots associated with this dataset.

In [ ]:
# Write our GraphQL query.
query = gql("""
query ($signal: String!) {
    all_signals (limit: 3, where: {name: {contains: $signal}}) {
        signals {
          name
          url
          shot {
            shot_id
            timestamp
            divertor_config
          }
        }
    }
}
""")
# Query the API and get a JSON response
client.execute(query, variable_values={"signal": "AMC"})

### Pagination in GraphQL

GraphQL queries are paginated. You may access other entries by including the page metadata and its associated elements. Here's an example of getting paginated entries:

In [ ]:
def do_query(cursor: str = None):
    query = gql("""
    query ($cursor: String) {
        all_shots (limit: 3, where: {campaign: {contains: "M9"}}, cursor: $cursor) {
            shots {
                shot_id
                timestamp
                preshot_description
                divertor_config
            }
            page_meta {
              next_cursor
              total_items
              total_pages
            }
        }
    }
    """)
    return client.execute(query, {'cursor': cursor})


def iterate_responses():
    cursor = None
    while True:
        response = do_query(cursor)
        yield response
        cursor = response['all_shots']['page_meta']['next_cursor']
        if cursor is None:
            return

responses = iterate_responses()
print(next(responses))
print(next(responses))
print(next(responses))